In [1]:
import time
import math
import copy
import torch
import pickle
import random
import logging
import warnings
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import torch.nn as nn
import torch.optim as opt
import matplotlib.pyplot as plt
import fairlearn.datasets
from tqdm import tqdm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from torch.autograd import grad
from torch.autograd.functional import vhp
from get_datasets import get_diabetes
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score
from scipy.stats import spearmanr

plt.rcParams['figure.dpi'] = 300
warnings.filterwarnings("ignore")

E = math.e

### Utility Functions

In [2]:
def visualize_result(e_k_actual, e_k_estimated, ep, k_):

    #e_k_estimated = [-1*ek for ek in e_k_estimated]
    fig, ax = plt.subplots()
    palette = sns.color_palette("cool", len(e_k_actual))
    sns.set(font_scale=1.15)
    sns.set_style(style='white')
    min_x = np.min(e_k_actual)
    max_x = np.max(e_k_actual)
    min_y = np.min(e_k_estimated)
    max_y = np.max(e_k_estimated)
    
    plt.rcParams['figure.figsize'] = 6, 5
    z = np.polyfit(e_k_actual,  e_k_estimated, 1)
    p = np.poly1d(z)
    xx = np.linspace(-p(2)/p(1), max(e_k_actual)+.0001)
    yy = np.polyval(p, xx)
    #add trendline to plot
    ax.plot(xx, yy, ls="-", color='k')
    for k in range(len(e_k_actual)):
        ax.scatter(e_k_actual[k], e_k_estimated[k], zorder=2, s=45, color = palette[k], label=ep[k])

    ax.set_title(f'Actual vs. Estimated loss for k={k_:.2f}%')
    ax.set_xlabel('Actual loss difference')
    ax.set_ylabel('Estimated loss difference')
   
    ax.set_xlim(min_x-.0001, max_x+.0001)
    ax.set_ylim(min_y-.0001, max_y+.0001)
    
   
    text = 'MAE = {:.03}\nP = {:.03}'.format(mean_absolute_error(e_k_actual, e_k_estimated), spearmanr(e_k_actual, e_k_estimated).correlation)
    ax.text(max_x+.00009,min_y-.00008, text, verticalalignment='bottom', horizontalalignment='right')
    #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #plt.tight_layout()
    plt.xticks(rotation = 45)
    plt.show()
    # cooler color = smaller epsilon

In [3]:
 class CreateData(torch.utils.data.Dataset):
    def __init__(self, data, targets, pert_status):
        self.data = data
        self.targets = targets
        self.pert = pert_status

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.targets[idx]
        pert_label = self.pert[idx]

        return out_data, out_label, pert_label

In [4]:
def get_data(new_train_df, feature_set, label, k):    

    selected_group = new_train_df.loc[new_train_df['gender'] == 0]

    num_to_sample = round((k / 100)*len(selected_group))

    sampled_group = selected_group.sample(n=num_to_sample)
    not_selected = new_train_df.drop(sampled_group.index)

    selected_group_X = sampled_group[feature_set]
    selected_group_y = sampled_group[label]

    not_selected_group_X = not_selected[feature_set]
    not_selected_group_y = not_selected[label]   
    
    return selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y

### Forward Loss Correction


In [5]:
def forward_correct_loss(y_act, oupt, epsilon, criterion, device):
    
    p = get_p(epsilon)
    y_pred_c = torch.matmul(oupt, p.to(device)) 
    loss = criterion(torch.log(y_pred_c), y_act)
    
    return loss 

### Randomized Response

In [6]:
def get_p(epsilon):
    probability = float(E ** epsilon) / float(1 + (E ** epsilon))
    p = torch.FloatTensor([[probability, 1-probability], [1-probability, probability]])
    
    return p

### Models

In [7]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        
        self.fc1 = torch.nn.Linear(num_features, 2, bias=False)
        self.criterion = torch.nn.NLLLoss(reduction='mean')
        self.activ = torch.nn.Softmax()
        
    def forward(self, x):
        logits = self.activ(self.fc1(x))

        return logits
    
    def loss(self, logits, y, print_):
        loss = self.criterion(torch.log(logits), y)
        
        return loss

In [8]:
def train(model, dataset, epsilon, lengths, device):
    model.train()
    
    opt = torch.optim.SGD(model.parameters(), lr=.0001, weight_decay=0)
    criterion = torch.nn.NLLLoss(reduction='mean')
            
    pert_status = np.zeros(len(dataset[0]))
    
    if lengths is not None:
        len_original = lengths[0]
        len_perts = lengths[1]
        total_len = len(dataset[0])
        pert_status = []
        pert_status.extend(np.zeros(len_original))
        pert_status.extend(np.ones(len_perts))
            
    train_data = CreateData(dataset[0], dataset[1], pert_status)
    train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)

    for itr in range(0, 10):
        itr_loss = 0
        correct = 0
        for i, [x,y,p] in enumerate(train_dataloader):
            opt.zero_grad()
            oupt = model(x)
            if p == 0:
                loss_val = criterion(torch.log(oupt), y)
            else:
                loss_val = forward_correct_loss(y, oupt, epsilon, torch.nn.NLLLoss(reduction='sum'), device)
            itr_loss += loss_val
            loss_val.backward()
            opt.step()
            
            _, predicted = torch.max(oupt.data, 1)
            correct+= (predicted == y.data).sum()
            
        print(f'Epoch {itr+1} - Loss: {itr_loss/len(train_dataloader):.3f} - Acc: {100*correct/len(train_dataloader):.3f}')
            
    return model

### Influence Calculation Functions


In [9]:
def calc_influence_single(model, epsilon, train_data, test_data, group_data, device, num_features, criterion):
    # criterion = mean
    start = time.time()
    est_hess = explicit_hess(model, train_data, device, criterion)

    grad_test = grad_z([test_data[0], test_data[1]], model, device, criterion)
    s_test_vec = torch.mm(grad_test[0], est_hess.to(device))

    P = get_p(epsilon)
    
    p_01, p_10 = P[0][1].item(), P[1][0].item()
    
    pi_1 = sum(list(group_data[1]))
    pi_0 = len(group_data[1]) - pi_1
    
    lam_0 = round(p_01 * pi_1)
    lam_1 = round(p_10 * pi_0)

    S_pert = 1 - group_data[1]
    
    y_w_group_pert = pd.concat([group_data[3], S_pert], axis = 0, ignore_index=True)
    y_wo_pert = pd.concat([group_data[3], group_data[1]], axis = 0, ignore_index=True)
    reconstructed_x = pd.concat([group_data[2], group_data[0]], axis = 0, ignore_index=True)
  
    assert len(S_pert) == len(group_data[1])
    grad_z_vec = grad_training([group_data[0],group_data[1]], S_pert, [model], device, [lam_0, lam_1, epsilon], criterion)
    
    influence = torch.dot(s_test_vec.flatten(), grad_z_vec[0].flatten()) * (1/len(train_data[0]))

    end = time.time() - start

    return influence.cpu(), end

In [10]:
def explicit_hess(model, train_data, device, criterion):
 
    pred_prob = model(train_data[0])
    loss = criterion(torch.log(pred_prob), train_data[1])
    
    grads = grad(loss, model.parameters(), retain_graph=True, create_graph=True)

    hess_params = torch.zeros(len(model.fc1.weight[0]), len(model.fc1.weight[0]))
    for i in range(len(model.fc1.weight[0])):
        hess_params_ = grad(grads[0][0][i], model.parameters(), retain_graph=True)[0][0]
        for j, hp in enumerate(hess_params_):
            hess_params[i,j] = hp
    
    inv_hess = torch.linalg.inv(hess_params)
    return inv_hess

In [11]:
def grad_z(test_data, model, device, criterion):

    model.eval()

    test_data_features = test_data[0]
    test_data_labels = test_data[1]

    pred_prob = model(test_data_features)
    loss = criterion(torch.log(pred_prob), test_data_labels)
    
    return grad(loss, model.parameters())

In [12]:
def grad_training(train_data, y_perts, parameters, device, epsilon, criterion):

    criterion = torch.nn.NLLLoss(reduction='sum')
    
    lam_0, lam_1, ep = epsilon
    lam = lam_0 + lam_1
    len_s = len(y_perts)

    train_data_features = torch.FloatTensor(train_data[0].values).to(device)
    train_data_labels = torch.LongTensor(train_data[1].values).to(device)
    train_pert_data_labels = torch.LongTensor(y_perts.values).to(device)
    
    model = parameters[0]
    model.eval()
    
    pred_prob = model(train_data_features)

    pert_loss = forward_correct_loss(train_pert_data_labels, pred_prob, ep, criterion, device)
    orig_loss = criterion(torch.log(pred_prob), train_data_labels)
    
    loss = float(1/(1 + (E ** ep)))*(pert_loss - orig_loss)
    to_return = grad(loss, model.parameters())

    return to_return

### Main Function

In [13]:
def Main(dataset, epsilons, ks, num_rounds):

    device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
    criterion = torch.nn.NLLLoss(reduction='mean')
    
    all_orig_loss_e_k = []
    all_est_loss_e_k = []
    all_time = []
    
    for nr in range(num_rounds):
        print(f'\nRound {nr+1}')
        ############
        # Get data #
        ############
        print('\nGetting Data...')
        if dataset == 'adult':
            data = get_adult()
            label = 'income_class'
        elif dataset == 'diabetes':
            data = get_diabetes()
            label = 'readmit_binary'
        else:
            data = get_law()
            label = 'admit'

        feature_set = set(data.columns) - {label}
        num_features = len(feature_set)
    
        X = data[feature_set]
        y = data[label]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
  
        new_train_df = pd.concat([X_train, y_train], axis=1)
  
        train_sample_num = len(X_train)
    
        x_test_input = torch.FloatTensor(X_test.values).to(device)
        y_test_input = torch.LongTensor(y_test.values).to(device)

        x_train_input = torch.FloatTensor(X_train.values).to(device)
        y_train_input = torch.LongTensor(y_train.values).to(device)
   
        ##############################################
        # Train original model and get original loss #
        ##############################################
        print('Training original model...')
        torch_model = LogisticRegression(num_features)
        torch.save(torch_model.state_dict(), 'initial_config_flc.pth')
        torch_model.to(device)
        torch_model = train(torch_model, [x_train_input, y_train_input], None, None, device)
        test_loss_ori = torch_model.loss(torch_model(x_test_input), y_test_input, True)
        
        e_k_act_losses = []
        e_k_est_losses = []
        influence_time = []
        
        ################################################################
        # Perform influence and retraining for all epsilons a k values #
        ################################################################
        print('\nBegining epsilon and k rounds')
        print('-----------------------------')
        for ep in epsilons:
            print(f'\nEpsilon: {ep}')
            
            k_act_losses = []
            k_est_losses = []
            inf_time = []
            
            for k in ks:
                
                # Influence
                print(f'k: {k:.2f}')
                
                selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y = get_data(new_train_df, feature_set, label, k)

                loss_diff_approx, tot_time = calc_influence_single(torch_model, ep, [x_train_input, y_train_input], [x_test_input, y_test_input], [selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y], device, num_features, criterion)
                loss_diff_approx = -torch.FloatTensor(loss_diff_approx).cpu().numpy()
                print(f'Approx diff: {loss_diff_approx:.5f}')
                # Retrain
                P = get_p(ep)

                p_01, p_10 = P[0][1].item(), P[1][0].item()

                pi_1 = sum(list(selected_group_y))
                pi_0 = len(selected_group_y) - pi_1

                lam_0 = round(p_01 * pi_1)
                lam_1 = round(p_10 * pi_0)

                S = pd.concat([selected_group_X, selected_group_y], axis=1, ignore_index=False)

                G0 = S[label][S[label].eq(1)].sample(lam_0).index
                G1 = S[label][S[label].eq(0)].sample(lam_1).index

                G = S.loc[G0.union(G1)]
                not_g = S.drop(G0.union(G1))
                
                G_pert = 1 - G[label]

                y_w_group_pert = pd.concat([not_selected_group_y, not_g[label], G_pert], axis = 0, ignore_index=True)
                y_wo_pert = pd.concat([not_selected_group_y, not_g[label], G[label]], axis = 0, ignore_index=True)
                reconstructed_x = pd.concat([not_selected_group_X, not_g[feature_set], G[feature_set]], axis = 0, ignore_index=True)

                model_pert = LogisticRegression(num_features)
                model_pert.load_state_dict(torch.load('initial_config_flc.pth'))
                model_pert.to(device)
                model_pert = train(model_pert, [torch.FloatTensor(reconstructed_x.values).to(device), torch.LongTensor(y_w_group_pert.values).to(device)], ep, [len(not_selected_group_y)+ len(not_g), len(G)], device)
                test_loss_retrain = model_pert.loss(model_pert(x_test_input), y_test_input, False)

                 # get true loss diff
                loss_diff_true = (test_loss_retrain - test_loss_ori).detach().cpu().item()
                print(f'True diff: {loss_diff_true:.5f}')
                k_act_losses.append(loss_diff_true)
                k_est_losses.append(loss_diff_approx)
                inf_time.append(tot_time)
            
            e_k_act_losses.append(k_act_losses)
            e_k_est_losses.append(k_est_losses)
            influence_time.append(inf_time)
            
        all_orig_loss_e_k.append(e_k_act_losses)
        all_est_loss_e_k.append(e_k_est_losses) 
        all_time.append(influence_time)
    
    return all_orig_loss_e_k, all_est_loss_e_k, all_time

### Perform Experiment 

#### Constants

In [14]:
epsilons = [.01, .02, .03, .04, .05, .06, .07, .08, .09, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
k = np.linspace(1, 25, 10)
rounds = 3

In [15]:
print(k)

[ 1.          3.66666667  6.33333333  9.         11.66666667 14.33333333
 17.         19.66666667 22.33333333 25.        ]


In [ ]:
all_orig_loss_e_k, all_est_loss_e_k, all_time = Main('diabetes', epsilons, k, rounds)
# [round1[epsilon1[k1,...k10], epsilon2[k1,...k10],...], round2[...]]     

with open('all_orig_loss_e_k_flc_dia.txt', "wb") as file:   #Pickling
    pickle.dump(all_orig_loss_e_k, file)

with open('all_est_loss_e_k_flc_dia.txt', "wb") as file2:   #Pickling
    pickle.dump(all_est_loss_e_k, file2)


Round 1

Getting Data...
Training original model...
Epoch 1 - Loss: 0.675 - Acc: 58.073
Epoch 2 - Loss: 0.659 - Acc: 61.430
Epoch 3 - Loss: 0.655 - Acc: 61.828
Epoch 4 - Loss: 0.654 - Acc: 61.973
Epoch 5 - Loss: 0.653 - Acc: 62.003
Epoch 6 - Loss: 0.653 - Acc: 62.057
Epoch 7 - Loss: 0.652 - Acc: 62.129
Epoch 8 - Loss: 0.652 - Acc: 62.095
Epoch 9 - Loss: 0.652 - Acc: 62.058
Epoch 10 - Loss: 0.652 - Acc: 62.096

Begining epsilon and k rounds
-----------------------------

Epsilon: 0.01
k: 1.00
Approx diff: -0.00004
Epoch 1 - Loss: 0.675 - Acc: 58.019
Epoch 2 - Loss: 0.659 - Acc: 61.385
Epoch 3 - Loss: 0.655 - Acc: 61.733
Epoch 4 - Loss: 0.654 - Acc: 61.848
Epoch 5 - Loss: 0.653 - Acc: 61.910
Epoch 6 - Loss: 0.653 - Acc: 61.956
Epoch 7 - Loss: 0.652 - Acc: 62.041


In [ ]:
with open('all_orig_loss_e_k_flc_dia.txt', 'rb') as f:
    all_orig_loss_e_k = pickle.load(f)
    
with open('all_est_loss_e_k_flc_dia.txt', 'rb') as f:
    all_est_loss_e_k = pickle.load(f)

In [ ]:
# [actual, estimate]

sum_orig_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_est_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_time = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]

avg_orig_loss = []
avg_est_loss = []
avg_time = []

for round_ in range(len(all_orig_loss_e_k)):
    for e in range(len(epsilons)):
        for k_ in range(len(k)):
            sum_orig_loss_e_k[e][k_] = sum_orig_loss_e_k[e][k_] + all_orig_loss_e_k[round_][e][k_]
            sum_est_loss_e_k[e][k_] = sum_est_loss_e_k[e][k_] + all_est_loss_e_k[round_][e][k_]
            #sum_time[e][k_] = sum_time[e][k_] + all_time[round_][e][k_]

for e in range(len(epsilons)):
    avg_orig_loss.append([ elem / len(all_orig_loss_e_k) for elem in sum_orig_loss_e_k[e]])
    avg_est_loss.append([elem/ len(all_orig_loss_e_k) for elem in sum_est_loss_e_k[e]])
    avg_time.append([elem/ len(all_orig_loss_e_k) for elem in sum_time[e]])

k_e_orig = [[] for _ in range(len(k))]
k_e_est = [[] for _ in range(len(k))]

for e in range(len(epsilons)):
    for k_ in range(len(k)):
        k_e_orig[k_].append(avg_orig_loss[e][k_])
        k_e_est[k_].append(avg_est_loss[e][k_])
        
print(k_e_orig)

averaged_time = []

for e in range(len(epsilons)):
    averaged_time.append(sum_time[e][0])

average_time_final = sum(averaged_time) / len(averaged_time)

In [ ]:
k = np.linspace(1, 25, 10)
k_ = list(k)
k_

In [ ]:
for i in range(len(k_e_orig)):
    visualize_result(k_e_orig[i], k_e_est[i], epsilons, k_[i])